In [160]:
# this part of the codes fetches requirements
import hashlib
from phe import paillier
import random as rd
import numpy as np

In [161]:
listOfIDS = []
dictionaryOfPublicKeys = {}
dictionaryOfPrivateKeys = {}
dictionaryOfRandomNumbers = {}
dictionaryOfBids = {}
finalSubmittedDataFirstPhase = [] #will be sused to stored the final resuklt of the two bidder for bid submission
highestbidder = 0 
numberOfBidders = 0 
publicKeyAuc, privateKeyAuc = 0, 0
finalDataOfSecondPhase = []
productOfAllRandomNumbers = 1

In [162]:
#  this part of the code creates unique BID for all participants
def createBiddersAndKeys(n):
  global numberOfBidders,dictionaryOfPublicKeys,dictionaryOfPrivateKeys,publicKeyAuc,privateKeyAuc
  numberOfBidders = n
  for i in range(0,numberOfBidders):
    i = str(i)
    listOfIDS.append(hashlib.sha256(i.encode('utf-8')).hexdigest())
  i = 0
  publicKeyAuc, privateKeyAuc = paillier.generate_paillier_keypair(n_length= 64)
  for i in range(0, numberOfBidders):
    x,y = paillier.generate_paillier_keypair(n_length= 128)
    dictionaryOfPublicKeys[listOfIDS[i]],dictionaryOfPrivateKeys[listOfIDS[i]]  = x ,y
  i = 0

end of the Registration phase and start of Bid Submission Phase

In [163]:
# this part of the code generate random number and distinct bid values for all bidders and Auctioneer
def  createRandomNumberAndBids():
    global dictionaryOfBids,dictionaryOfRandomNumbers
    bids  = rd.sample(range(1, numberOfBidders+1 ), numberOfBidders)
    for i in range(0,numberOfBidders):
        r  = rd.randint(100,10000)
        dictionaryOfRandomNumbers[listOfIDS[i]] =  r
        dictionaryOfBids[listOfIDS[i]] = bids[i]
    i = 0
    print(dictionaryOfBids)
    print(dictionaryOfRandomNumbers)


In [164]:
# this PART OF THE CODE SIMULATES BIDDER GENERATING PRODUCT AND THAN SENDING TO NEXT BIDDER

def initFirstPhase():
    global finalSubmittedDataFirstPhase
    for i in range(0,numberOfBidders):
        id = listOfIDS[i]
        dataPerBidder= []
        # print(dictionaryOfBids[id] * dictionaryOfRandomNumbers[id])
        encrytionOfBidAndRandomNumber = publicKeyAuc.raw_encrypt(dictionaryOfBids[id] * dictionaryOfRandomNumbers[id])
        # this part of code is forwarding the encrytionOfBidAndRandomNumber to all other bidders and after multiplication of the random number of the next
        # bidder it is sent to the Auctioneer immediately
        for j in range(0,numberOfBidders):
            if j != i:
                idOfNextBidder = listOfIDS[j]
                pubKeyNextBidder, priKeyNextBidder= dictionaryOfPublicKeys[idOfNextBidder],dictionaryOfPrivateKeys[idOfNextBidder]
                e = pubKeyNextBidder.raw_encrypt(encrytionOfBidAndRandomNumber)
                e = priKeyNextBidder.raw_decrypt(e)
                encrytionOfBidAndBothRandomNumber = e ** dictionaryOfRandomNumbers[idOfNextBidder]
                # this part of the code simulates the storing of the data of the bid summision to auctioneer data
                dataPerBidder.append(privateKeyAuc.raw_decrypt(encrytionOfBidAndBothRandomNumber))
            else:
                dataPerBidder.append(0)
        finalSubmittedDataFirstPhase.append(dataPerBidder)
    print(finalSubmittedDataFirstPhase)
    i,j = 0,0

implementation of the second phase here auctioneer reforward each packet to every bidder to multiply their random number.

In [165]:
# #this function works as the intermediate steps where we are requesting each bidder to multiplyt his random number to the given ciphertext
# def productOfRandomNumbers():
#     global productOfAllRandomNumbers
#     for k in range(numberOfBidders):
#         productOfAllRandomNumbers = productOfAllRandomNumbers * dictionaryOfRandomNumbers[listOfIDS[k]]


In [166]:
def everyBiddersRandomNumberMultiplication(encryptedAuctioneerRandomNumber):
    global productOfAllRandomNumbers
    print("////")
    for i in range(numberOfBidders):     
        temp = pow(encryptedAuctioneerRandomNumber,dictionaryOfBids[listOfIDS[i]])
        productOfAllRandomNumbers = temp * productOfAllRandomNumbers
        encryptedAuctioneerRandomNumber = productOfAllRandomNumbers 
        
    return productOfAllRandomNumbers

In [167]:
def initSecondPhase():
    global finalDataOfSecondPhase
    auctioneerRandomNumber = rd.randint(2,2)
    print(auctioneerRandomNumber)
    for i in range(len(finalSubmittedDataFirstPhase)):
        dataPerBidder = []
        for j in range(numberOfBidders):
            if finalSubmittedDataFirstPhase[i][j] == 0 :
                dataPerBidder.append(0)
            else:
                encryptedValueByAuctioneer = publicKeyAuc.raw_encrypt(auctioneerRandomNumber)
                returnValue = everyBiddersRandomNumberMultiplication(encryptedValueByAuctioneer)
                dataPerBidder.append((privateKeyAuc.raw_decrypt(returnValue)))
        finalDataOfSecondPhase.append(dataPerBidder)
    i,j = 0,0


In [168]:
# def runSimulation():
#     print("//")
#     createBiddersAndKeys(2)
#     print("//")
#     createRandomNumberAndBids()
#     print("//")
#     initFirstPhase()
#     print("//")
#     initSecondPhase()
#     print("//")

In [169]:
createBiddersAndKeys(10)
createRandomNumberAndBids()



{'5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91b46729d73a27fb57e9': 2, '6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d49c01e52ddb7875b4b': 9, 'd4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f90da3a666eec13ab35': 4, '4e07408562bedb8b60ce05c1decfe3ad16b72230967de01f640b7e4729b49fce': 1, '4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328cb08b5531fcacdabf8a': 3, 'ef2d127de37b942baad06145e54b0c619a1f22327b2ebbcfbec78f5564afe39d': 5, 'e7f6c011776e8db7cd330b54174fd76f7d0216b612387a5ffcfb81e6f0919683': 6, '7902699be42c8a8e46fbbb4501726517e86b22c56a189f7625a6da49081b2451': 8, '2c624232cdd221771294dfbb310aca000a0df6ac8b66b696d90ef06fdefb64a3': 7, '19581e27de7ced00ff1ce50b2047e7a567c76b1cbaebabe5ef03f7c3017bb5b7': 10}
{'5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91b46729d73a27fb57e9': 7815, '6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d49c01e52ddb7875b4b': 3026, 'd4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f90da3a666eec13ab35': 3403, '4e07408562bedb8b60ce05c1decfe3ad16b72230967de01f640b7e4729b49fce

In [170]:
initFirstPhase()

[[0, 47296380, 53188890, 117490710, 81229110, 16239570, 70897680, 118365990, 145062030, 64864500], [212833710, 0, 92677302, 204717978, 141535098, 28296126, 123533424, 206243082, 252758754, 113021100], [106377780, 41189912, 0, 102321404, 70741564, 14142868, 61744032, 103083676, 126332972, 56489800], [58745355, 22746442, 25580351, 0, 39065849, 7810163, 34097112, 56926241, 69765277, 31195550], [121843665, 47178366, 53056173, 117197547, 0, 16199049, 70720776, 118070643, 144700071, 64702650], [40598925, 15720070, 17678585, 39050815, 26998415, 0, 23564520, 39341735, 48214795, 21559250], [212693040, 82355616, 92616048, 204582672, 141441552, 28277424, 0, 206106768, 252591696, 112946400], [473463960, 183327184, 206167352, 455409928, 314855048, 62946776, 274809024, 0, 562280104, 251423600], [507717105, 196590142, 221082701, 488356939, 337633499, 67500713, 294690312, 491995091, 0, 269613050], [324322500, 125579000, 141224500, 311955500, 215675500, 43118500, 188244000, 314279500, 385161500, 0]]


In [171]:
# initSecondPhase()

In [172]:
random = publicKeyAuc.raw_encrypt(2)
output = 1
for i in range(5):
    output  = pow(output,dictionaryOfBids[listOfIDS[i]])
print(privateKeyAuc.raw_decrypt(output))   

0


In [173]:

for i in range(len(finalDataOfSecondPhase)):
    print(finalDataOfSecondPhase[i])
i = 0

In [174]:
for i in range(len(finalSubmittedDataFirstPhase)):
    print(finalSubmittedDataFirstPhase[i])
i = 0

[0, 47296380, 53188890, 117490710, 81229110, 16239570, 70897680, 118365990, 145062030, 64864500]
[212833710, 0, 92677302, 204717978, 141535098, 28296126, 123533424, 206243082, 252758754, 113021100]
[106377780, 41189912, 0, 102321404, 70741564, 14142868, 61744032, 103083676, 126332972, 56489800]
[58745355, 22746442, 25580351, 0, 39065849, 7810163, 34097112, 56926241, 69765277, 31195550]
[121843665, 47178366, 53056173, 117197547, 0, 16199049, 70720776, 118070643, 144700071, 64702650]
[40598925, 15720070, 17678585, 39050815, 26998415, 0, 23564520, 39341735, 48214795, 21559250]
[212693040, 82355616, 92616048, 204582672, 141441552, 28277424, 0, 206106768, 252591696, 112946400]
[473463960, 183327184, 206167352, 455409928, 314855048, 62946776, 274809024, 0, 562280104, 251423600]
[507717105, 196590142, 221082701, 488356939, 337633499, 67500713, 294690312, 491995091, 0, 269613050]
[324322500, 125579000, 141224500, 311955500, 215675500, 43118500, 188244000, 314279500, 385161500, 0]
